In [1]:
import matplotlib.pyplot as plt
from gerrychain import Graph, Partition, GeographicPartition, proposals, updaters, constraints, accept, MarkovChain, Election
from gerrychain.constraints import single_flip_contiguous
from gerrychain.updaters import cut_edges, Tally
from gerrychain.proposals import recom, propose_random_flip
from gerrychain.accept import always_accept
from functools import partial
import time
import geopandas as gpd

In [2]:
gdf = gpd.read_file("./SC/SC.shp")

In [3]:
graph = Graph.from_geodataframe(gdf)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gerrychain/graph/graph.py:179: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()


In [4]:
tot_pop = sum([graph.nodes()[v]['TOTPOP'] for v in graph.nodes()])
print(tot_pop)

5118425


In [5]:
initial_partition = GeographicPartition(graph=graph, 
                                        assignment="CD",
                                       updaters={
                                            "population": Tally("TOTPOP", alias="population"),
                                            "cut_edges": cut_edges,
                                            "democratic_votes": Tally("G20PRED", alias="democratic_votes"),
                                            "republican_votes": Tally("G20PRER", alias="republican_votes"),
                                            "latino_pop": Tally("HISP", alias="latino_pop")
    })

In [6]:
num_dist = len(initial_partition)
print(num_dist)
ideal_pop = tot_pop / num_dist
pop_tolerance = 0.07

7


In [7]:
rw_proposal = partial(recom,
                      pop_col = "TOTPOP",
                      pop_target = ideal_pop,
                      epsilon = pop_tolerance,
                      node_repeats = 1
                      )

In [11]:
population_constraint = constraints.within_percent_of_ideal_population(
    initial_partition,
    pop_tolerance,
    pop_key="population")

In [14]:
chain = MarkovChain(
    proposal = rw_proposal,
    constraints = [population_constraint],
    accept = always_accept,
    initial_state = initial_partition,
    total_steps = 10
)

ValueError: The given initial_state is not valid according is_valid. The failed constraints were: Bounds(population,(716579.5, 745827.642857143))

In [ ]:
cutedge_ensemble = []
latino_majority_ensemble = []
democratic_win_ensemble = []

In [ ]:
for part in chain:
    # Add cutedges to cutedges ensemble
    cutedge_ensemble.append(len(part['cut_edges']))

    # Calculate number of latino-majority districts
    # Add to ensemble
    num_maj_latino = 0
    for i in range(1, 8):
        latino_perc = part['latino_pop'][str(i)] / part['population'][str(i)]
        if latino_perc >= 0.5:
            num_maj_latino = num_maj_latino + 1
    latino_majority_ensemble.append(num_maj_latino)

    num_democratic_win = 0
    for i in range(1, 8):
        dem_perc = part['democratic_votes'][str(i)] / (
                    part['democratic_votes'][str(i)] + part['republican_votes'][str(i)])
        if dem_perc > 0.5:
            num_democratic_win = num_democratic_win + 1
    democratic_win_ensemble.append(num_democratic_win)

In [ ]:
print(cutedge_ensemble)
print(latino_majority_ensemble)
print(democratic_win_ensemble)